# Лекция 1 
# Базовый SQL 

In [ ]:
!pip install ipython-sql

In [ ]:
%load_ext sql
%sql sqlite://

In [ ]:
%%sql 
select 'Hello, world';

In [ ]:
%%sql 
select 'Hello, world' as hw

Пара замечаний:
 * Мы используем веб-интерфейс для python, поэтому для запуска SQL запросов необходимо применить ряд вещей:
     * Необходимо подключить расширение SQL через так называемые magic command. [Более подробно здесь](https://ipython.readthedocs.io/en/stable/interactive/magics.html)
     * Для работы с SQL надо вызвать либо %sql для однострочной команды, либо %%sql - для многострочной
     * При использовании выражения SELECT результатом вывода является таблица, но при этом в notebook выводится уже внутренне представление языка python (объект класса sql.run.resultset, подробности далее), что приводит к ряду несоответствий (например, None вместо NULL)

Давайте создадим таблицу, заполним ее и сделаем какой-нибудь запрос!

In [ ]:
%%sql                                             -- Базовый синтаксис:
drop table if exists product;                     -- удаление таблицы, если существует

create table product(                             -- синтаксис создания таблицы, объявление столбцов:
       pname        varchar primary key,          -- имя продукта, первичный ключ
       price        decimal(10,2),                -- цена продукта
       category     varchar(30),                  -- категория
       manufacturer varchar(30) NOT NULL          -- производитель, not null не дает вводить значение null
);

In [ ]:
%%sql
insert into product (pname, price, category, manufacturer) values ('Тетрадь', 39.99, 'Канцелярия', 'Академия холдинг');

insert into product values('Клавиатура', 949.99, 'Техника', 'Sven');

insert into product 
    values ('Степлер', 129.99, 'Канцелярия', 'Brauberg'),
           ('Батарейка', 39.99, 'Для дома', 'Energizer'),
           ('Энергетик', 89.00, 'Напитки', 'Redbull');

In [ ]:
%%sql
insert into product (pname, category) values ('Маркер', 'Канцелярия');
insert into product (pname, price, category, manufacturer) values ('Тетрадь', 49.99, 'Канцелярия', 'BG');

Посмотрим на полученную таблицу.

In [ ]:
%sql select * from product;

In [ ]:
product_result=%sql select * from product

In [ ]:
print(type(product_result))
print(product_result)

Обратите внимание на тип. Более подробно по данной библиотеки можно почитать:
https://github.com/catherinedevlin/ipython-sql

# Немного SQL терминологии
* _имя_ таблицы - product.
* Каждая строка таблицы называется _строкой_ или _кортеж_. 
* Заметьте, что все кортежи имеют _поля_ или _атрибуты_.
* Количество строк называет _мощностью_, в то время как количество атрибутов _арностью_ 

# Простые запросы
* Рассмотрим основы SQL на примере
* В интернете очень много SQL туториалов, сейчас мы всего лишь вспомним базовые вещи

> SELECT <Множество атрибутов><br>
> FROM <список таблиц и условие на соединение><br>
> WHERE <список условий>

Это простейший SELECT-FROM-WHERE (SFW) блок. Давайте посмотрим на примерах!

In [ ]:
%%sql
SELECT * 
  FROM Product 
WHERE category = 'Канцелярия' AND manufacturer = 'Brauberg';

\* \- В данном случае обозначает вывод всех полей, которые были описаны во FROM

Посмотрим на примеры *проекции*, то есть получим только несколько атрибутов запроса.

In [ ]:
%%sql 
SELECT Pname, 
       Price, 
       Manufacturer
  FROM Product;

* На выходе *все еще* таблица и ее схема -
> Answer(pname, price, manufacturer)

* Можно объединять выборку и проекцию + менять результат

In [ ]:
%%sql
SELECT Pname, Price * 2 as price_x2, Manufacturer
FROM Product
WHERE category in ('Напитки', 'Канцелярия');

In [ ]:
%%sql
SELECT pname || '->' || price || '->' || manufacturer as concat
FROM Product
WHERE category = 'Напитки';

На выходе запроса к таблице - снова таблица 
---------------------------------

In [ ]:
%%sql
SELECT p0.manufacturer
FROM Product p0
WHERE p0.price < 100.00;

In [ ]:
%%sql
SELECT
    p.manufacturer, p.pname, p.price
FROM 
    (SELECT p0.manufacturer
     FROM Product p0
     WHERE p0.price < 100.00) cp, -- вложенный запрос
    Product p
WHERE 
    p.manufacturer = cp.manufacturer AND p.category = 'Канцелярия'

Небольшие детали
--------------

* Некоторые элементы регистро-независимые:
  * Одно и то же: SELECT  Select  select
  * Одно и то же: Product   product
  * Разные: ‘Brauberg’  ‘brauberg’ (Здесь это строковая константа)



In [ ]:
%sql select * from product;

LIKE
====
Регулярные выражения

Опертор LIKE нужен для поиска строк:
    
> SELECT *
> FROM Products
> WHERE pname like '%word%'

* % - сколько угодно символов
* \_ ровно один символ
* оператор LIKE - регистрозависимый

In [ ]:
%%sql
SELECT * FROM product
where category LIKE '%ка%'

In [ ]:
%%sql
SELECT * FROM product
where category LIKE '_а%'

Убрать дубли
---------------------
* Дубли не всегда хорошо, и иногда их стоит убирать
  * Помните, что таблицы - _мультимножества_!

In [ ]:
%sql SELECT category from product;

In [ ]:
%%sql 
-- чтобы убрать дубли используйте слово DISTINCT
SELECT DISTINCT category from product;

Сортировка результатов
---------------------
* Так как Таблица - это мультимножество, то порядок вывода строк не гарантирован. Иногда необходимо выводить строки в определенном порядке


In [ ]:
%%sql
-- сортировка результатов
-- сортировка по умолчанию - ascending
SELECT   pname, price, manufacturer
FROM     Product
WHERE    price < 400
ORDER BY price, pname

In [ ]:
%%sql
-- сортировка результатов
-- тип сортировки каждого компонента определяется индивидуально
SELECT   price, manufacturer
FROM     Product
ORDER BY   manufacturer ASC, price DESC

Можно делать сортировку по порядковому номеру, но довольно часто это считается bad practice 

In [ ]:
%%sql
SELECT   price, manufacturer
FROM     Product
ORDER BY   2 ASC, 1 DESC

Работа с несколькими таблицами
------


* Рассмотрим таблицу компаний.
> company(<u>cname</u>, stockprice, country)
 

In [ ]:
%%sql
drop table if exists product;
drop table if exists company;

create table company (
    cname varchar primary key,    -- Имя
    production varchar,           -- Продукция
    country varchar);             -- Страна

insert into company values ('Brauberg', 'Товары для офиса', 'Россия');
insert into company values ('Energizer', 'Аккумуляторы', 'США');
insert into company values ('Sven', 'Акустическая система и компьютерная периферия', 'Финляндия');

In [ ]:
%sql select * from company;

Внешние ключи
-----------------------
* Допустим мы ходим добавить таблицу продуктов

> Product(pname, price, category, manufacturer)

* Может возникнуть следующая ситуация: есть компания, продающая какие-то продукты, но при этом она отсутствует в таблице компаний.
* Чтобы избежать это, воспользуемя _внешними ключами_ 

Назвнание компании в product _ссылается_ на поле cname из таблицы company:

> foreign key (manufacturer) references company(cname)

  * Замечание: cname <u>должно быть</u> ключом в  company! 
  

In [ ]:
%%sql 

drop table if exists product;

pragma foreign_keys = ON; -- WARNING by default off in sqlite

create table product(
       pname        varchar primary key,  -- имя продукта
       price        money,                -- цена продукта
       category     varchar,              -- категория
       manufacturer varchar NOT NULL,      -- производитель
       foreign key (manufacturer) references company(cname)
);

insert into product values('Клавиатура', 949.99, 'Техника', 'Sven');
insert into product values('Наушники', 1999.99, 'Техника', 'Sven');
insert into product values('Степлер', 129.99, 'Канцелярия', 'Brauberg');
insert into product values('Маркер', 59.99, 'Канцелярия', 'Brauberg');
insert into product values('Батарейка', 39.99, 'Для дома', 'Energizer');

Внешние ключи являются _ограничениями_ таблицы 
> Что случится при вставке компании, которой не в таблице company?


In [ ]:
try:
    %sql insert into product values('Тетрадь', 39.99, 'Канцелярия', 'Академия холдинг');
except Exception as e:
    print (e)
    print ("Rejected!")

In [ ]:
%%sql
-- the update is rejected!
select * from product;

In [ ]:
%%sql 
select * from company;

Внешние ключи и удаление
=============

* Что произойдет, если мы удалим компанию ? Несколько вариантов:
  * Запретить удаление (default)
  * Удалить все продукты (добавьте "`on delete cascade`")
  * Замена на  NULL или DEFAULT
  

**Первый вариант (default)- Удаление запрещено**

In [ ]:
try:
    %sql delete from company where cname = 'Brauberg';
except Exception as e:
    print (e)
    print ("Rejected!")

**Второй вариант: удалить все продукты, принадлежащие компании, которую мы удаляем**

Необходимо изменить опеределение в create table:
> foreign key (manufacturer) references company(cname) on delete cascade

Теперь, когда строка компании удалена, все продукты с указанной компанией, будут также удалены.

Соединения
------
> Product (<u>pname</u>,  price, category, manufacturer)<br>
> Company (<u>cname</u>, production, country)

Мы хотим ответить на вопрос

> Найти все продукты меньшие 100р, произведенные в России

> Возвратить их имена и цену. 



In [ ]:
%%sql
SELECT pname, price
FROM product, company
where manufacturer=cname and country='Россия' and price < 100;

Другой вариант написания запроса.

In [ ]:
%%sql -- Часть 1: выбрать все компании из России
SELECT distinct cname -- нужна ли нам уникальность?
from company where country='Россия';

In [ ]:
%%sql -- Часть 2: Продукты до 100
select pname, price, manufacturer
from product
where price <= 100;

In [ ]:
%%sql 
-- Объединение как cross join
SELECT * 
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 100) CheapProducts,
  (SELECT DISTINCT cname
   FROM company
   WHERE country='Россия') RusProducts;

In [ ]:
%%sql
-- Фильтруем cross join
SELECT DISTINCT pname, price
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 100) CheapProducts,
  (SELECT distinct cname
   FROM company
   WHERE country='Россия') RusProducts
WHERE cname = manufacturer;

# Использование конструкции JOIN

Есть другой вариант записи - через конструкцию JOIN. Кроме этого она позволяет делать также внешние запросы.



In [ ]:
%%sql
SELECT *
  FROM product p 
  JOIN company c 
    ON p.manufacturer = c.cname
 WHERE c.country='Россия' 
   AND p.price <= 100;

Примечания
--------
* Есть множество логических вариантов написать один и тот же запрос
    * Этот факт будет использоваться для оптимизации

Дубли после соединения
--------------------------

Замечание - могут возникнуть дубли после соединения...

In [ ]:
%%sql
SELECT Country
FROM Product, Company
WHERE manufacturer=cname AND category='Канцелярия';

# Группировка 


Можно осуществлять схлопывание строк в рамках каких-нибудь групп

После схлопывания можно применять так называемые агрегатные функции

AVG(<поле>) 

SUM(<поле>)

MIN(<поле>)

MAX(<поле>)

COUNT(<поле>)

COUNT(*)

AVG(distinct <поле>)

SUM(distinct <поле>)

COUNT(distinct <поле>)

In [ ]:
%%sql 
select category, avg(price), min(price), max(price), sum(price)
from Product
group by category

Существует возможность фильтровать не только по строкам, но и по группам. Для этого можно использовать инструкцию HAVING

In [ ]:
%%sql 
select category, avg(price), min(price), max(price), count(price)
from Product
group by category
having min(price) < 100

In [ ]:
%%sql 
select category, count(*) as star, count(category) as cat, count(distinct category) as dist
from Product
group by category;

count(*) - кол-во строк

count(<поле>) - кол-во непустых строк

count(distinct <поле>) - кол-во уникальных непустых строк

In [ ]:
%%sql 
drop table if exists numbers;

create table numbers(a int, b int);

insert into numbers 
  values (1,2), (2,3), (4,null), (null,null), (null,5);

In [ ]:
%sql select * from numbers;

In [ ]:
%%sql 
select sum(a+b) as sum1,
       sum(a) + sum(b) as sum2 
       from numbers;

In [ ]:
%%sql 
select a || '+' || b as concat
       from numbers;

In [ ]:
%%sql 
select *
  from numbers
where a>0;